In [1]:
from scrapping import get_data
from collections import Counter
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import itertools

In [2]:
db = pd.read_parquet('db.parquet')
winners = db.iloc[:, 1:7].set_index(['Sorteos'])
df_0 = pd.DataFrame(columns = [str(i) for i in range(1, 51)], index = [0]).fillna(True)
winners = pd.concat([df_0, winners], ignore_index = True).reset_index(drop = True)

for e in range(1, 6):
    for i in range(1, 51):
        winners.loc[winners[f"Nro{e}"] == i, f"{i}"] = True

winners.drop(columns = ["Nro1","Nro2","Nro3","Nro4","Nro5"], inplace = True)
winners.drop(columns = winners.iloc[:, 50:100], inplace = True)

In [5]:
winners

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,True
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,True,NaN,NaN,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,NaN,True,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,True,NaN,NaN,True,NaN,NaN,NaN
1606,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN
1607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,True,NaN,NaN,True,NaN,NaN,NaN,NaN
1608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,...,NaN,True,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN


In [3]:
draws = list(np.arange(1, len(winners)))
# c = [str(i) for i in range(1, 51)]
# dicts = {draw: {key:[] for key in c} for draw in draws}
# contador = 0
# e = 0

# while e != len(winners) - 1:
#     for columns in winners:
#         df = winners.loc[:draws[e]]
#         for i in reversed(df[columns]):
#             if i is not True:
#                 contador += 1
#             else:
#                 dicts[draws[e]].update({columns:contador})
#                 contador = 0
#                 break
#     e += 1

# skip_numbers = pd.DataFrame.from_dict(dicts, orient = 'index')

In [11]:
print(winners)

         1     2     3     4     5     6     7     8     9    10  ...    41  \
0     True  True  True  True  True  True  True  True  True  True  ...  True   
1      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...  True   
2      NaN   NaN   NaN   NaN   NaN   NaN  True   NaN   NaN   NaN  ...   NaN   
3      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   NaN   
4      NaN   NaN   NaN  True   NaN   NaN  True   NaN   NaN   NaN  ...   NaN   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
1605   NaN  True   NaN   NaN   NaN   NaN   NaN  True   NaN   NaN  ...   NaN   
1606   NaN   NaN  True   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   NaN   
1607   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   NaN   
1608   NaN   NaN   NaN   NaN   NaN   NaN   NaN  True   NaN   NaN  ...   NaN   
1609   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...  True   

        42    43    44    45    46    47    48    4

In [9]:
dicts = {draw: {key: sum(1 for _ in group) for key, group in itertools.groupby(winners.loc[:draws[draw]]['1'])} for draw in draws}

IndexError: list index out of range

In [ ]:
last_draw = [t[::-1] for t in sorted(dicts[len(skip_numbers)].items(), key = lambda x: x[1])]
last_draw = [(a, int(b)) for a, b in last_draw]
ld_skips = pd.DataFrame(last_draw).groupby(0)[1].apply(list).reset_index().rename({0: 'Skip', 1: 'Numbers'}, axis = 1)
x_12 = np.arange(len(skip_numbers) - 12, len(skip_numbers) + 1)
sk_12 = skip_numbers.loc[x_12]
skips = np.arange(0, 19)
aus = []
last_7 = []
last_12 = []

for i in x_12[1:13]:
    for column in sk_12:
        if sk_12.loc[i, str(column)] == 0:
            aus.append(sk_12.loc[i - 1, str(column)])

for number in skips:
    last_12.append(aus.count(number))
    last_7.append(aus[25:60].count(number))
    
skips_7_12 = pd.DataFrame(np.column_stack([last_7, last_12]), columns = [7, 12])

In [ ]:
L = groups_df.values.tolist()
groups_df = db. loc[x_12[2:12]]
groups_df.drop(['Dates', 'Sorteos', 'Star_1', 'Star_2'], axis = 1, inplace = True)
Counter(x//10 for x in L)
sorted(Counter(x//10 for x in L).items())

for k, v in sorted(Counter(x//10 for x in L).items()):
    print("%d - %d: %d"%(k*10, k*10+9, v))

In [ ]:
ld_skips.loc[2, 'Numbers'][0]

In [ ]:
# values = list(set(val for dic in [ultimo_sorteo] for val in dic.values()))